In [1]:
from aoc_utils import *

import pandas as pd
import numpy as np

from aocd import get_data, submit

In [2]:
year = 2024
day = 16

In [33]:
dat = get_data(day=day, year=year, block=True)

In [25]:
dat = '''###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############'''

In [34]:
g = Grid(parse(dat))
for k,v in g.items():
    if v == 'E':
        goal = k
    if v == 'S':
        init = (k, East)

class ReinProblem(GridProblem):
    def actions(self, state):
        loc, dir = state
        return [(n,sub(n,loc)) for n in self.grid.neighbors(loc) if g[n] != '#']
    def action_cost(self, s, a, s1):
        loc, dir = s
        nloc, ndir = a
        if dir == (0,0):
            return 1
        elif dir == ndir:
            return 1
        elif add2(dir, ndir) == (0,0):
            return np.inf
        else:
            return 1001
    def is_goal(self, state):
        return state[0] == self.goal
    def h(self, node):
        loc = node.state[0]
        return taxi_distance(loc, self.goal)

gp = ReinProblem(grid=g, initial=init, goal=goal)

p1 = A_star_search(gp)
p1.path_cost

72400

In [20]:
g2 = g.copy()
dirs = {North:'^', South:'v', East:'>',West:'<'}
for s in path_states(p1):
    g2[s[0]] = dirs.get(s[1], 'S')
g2.print()

###############
#.......#....^#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#....^>>>>>>#^#
###.#^#####v#^#
#...#^....#v#^#
#.#.#^###.#v#^#
#^>>>>#...#v#^#
#^###.#.#.#v#^#
#S..#.....#v>>#
###############


In [5]:
submit(p1, day=day, part='a', year=year)

Part a already solved with same answer: 56042


In [35]:

def A_star_search2(problem, h=None):
    """Search nodes with minimum f(n) = path_cost(n) + h(n) value first."""
    h = h or problem.h
    return best_first_search2(problem, f=lambda n: n.path_cost + h(n))

def best_first_search2(problem, f) -> 'Node':
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    soln_paths = []
    soln_cost = np.inf
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            if not soln_paths:
                soln_paths.append(node)
                soln_cost = node.path_cost
            else:
                # print(soln_cost, node.path_cost)
                if soln_cost < node.path_cost:
                    return soln_paths
                else:
                    soln_paths.append(node)
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost <= reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return search_failure


gp = ReinProblem(grid=g, initial=init, goal=goal)

p2 = A_star_search2(gp)

res = set()
for n in p2:
    for p,d in path_states(n):
        res.add(p)
len(res)

435

In [8]:
submit(p2, day=day, part='b', year=year)

Part b already solved with same answer: 55358
